In [1]:
%run "./ADP_Load_Conf"

In [2]:
################################################################################
"""Blob Utils.

"""
# Who                 When           What
# Victor Salesa       06/02/2019     Initial Version
# Victor Salesa       19/02/2019     TZBIADP-201 ERROR CONTROL SYSTEM: VALIDATION FILE NAME PROCESS NormalizeFarmaticSellout : 
#                                       - Added blob_delete_file_sql_lambda to Encapsulate delete function to avoid exception
# Victor Salesa       04/03/2019     Deleted password that was moved to ADP_Load_Conf 
# Victor Salesa       21/03/2019     blob_move_file & blob_copy_file Adding check for file copied.
# Victor Salesa       21/03/2019     blob_move_file & blob_copy_file Add timeout parameter default 1 sec
################################################################################

from azure.storage.blob import BlockBlobService
from pyspark.sql import Row
from fnmatch import fnmatch
from collections import namedtuple
from pyspark.sql.types import BooleanType
from pyspark.sql.types import IntegerType

#Structure to hold copy properties from blob storage
BlobCopyProperties = namedtuple("BlobCopyProperties", "source_container dest_container source_blob_name dest_blob_name")

#Current mount points
__BLOB_MOUNT_POINTS__     = [mount_point.mountPoint+'/' for mount_point in dbutils.fs.mounts() if 'wasbs://' in mount_point.source]

############################################################################################################################################################

def blob_ls(path,blob_mount_point_root=__BLOB_MOUNT_POINT_ROOT__,blob_account_name=__BLOB_ACCOUNT_NAME__,blob_account_key=__BLOB_ACCOUNT_KEY__,debug=False):
  """Simulates directory listing of azure blob storage container / virtual directory
  
      Parameters:
      path : string
        Path to be listed. It should follow the following pattern [OPTIONAL 'dbfs:'] __BLOB_MOUNT_POINT_ROOT__ / <CONTAINER_NAME / [OPTIONAL subdirectories] / [OPTIONAL filename or filename pattern(* or ?)]
         examples:
           'dbfs:/mnt/blob/landing-prod/pharmatic'
           ''/mnt/blob/landing-prod/pharmatic'
      blob_mount_point_root : string(s)
        Root of the mount points of blobs
          example:
            '/mnt/blob'
      blob_account_name : string
        Account Name to connect to blob storage
      blob_account_key
        Account Key to connect to blob storage
      debug: boolean
        Flag to include debug information or Not
      
      Return: Array(pyspark.sql.Row)
        Array of pyspark Rows with the following columns path,name,size,etag,last_modified of the data in the folder
        
  """
  #Who                 When           What
   #Victor Salesa Sanz  06/02/2019    Initial version
  ################################################################################

  try:
    raw_path = path.replace('dbfs:/','/')

    # Normalize with folder slash ending to avoid undesired effects
    if raw_path[-1:] !='/':
      raw_path = raw_path + '/'

    # Normalize with dbfs (All the mounts are there)
    path = 'dbfs:' + raw_path

    # Check if folder root is in BLOB mount points list
    is_in_mount_point = __builtin__.max([(1 if raw_path.startswith(mount_point) else 0) for mount_point in __BLOB_MOUNT_POINTS__ ])

    # Check if path contains blob mount point and creates a variable to retrieve container and prefix
    if is_in_mount_point==1:
      raw_path_container_blobs = raw_path.replace(blob_mount_point_root,'')
    else:
      raise Exception("blob_ls: Path does not contain a blob connection: "+path+",should be one of: ["+(",".join(__BLOB_MOUNT_POINTS__)) +"]" )

    # In previous string with container and prefix splits by '/' symbol to detect if a container name or prefix is present. 
    split_container_prefix  = raw_path_container_blobs.split('/')

    # Raises an exception if no container defined and extracts it if present
    if len(split_container_prefix) >= 1 and len(split_container_prefix[0])>0:
      
      # First value splitted by '/' should be container
      container = split_container_prefix[0]
    else:
      raise Exception("blob_ls: No container defined in the path "+path)

    #Gets the prefix by removing from raw path the base_blob_path and container
    prefix = raw_path.replace(blob_mount_point_root+container,'')

    #Removes the leading '/' symbol to match blob service standard
    if prefix[0]=='/':
      prefix = prefix[1:]

    #Connects to blob service
    block_blob_service = BlockBlobService(account_name=blob_account_name, account_key=blob_account_key)

    if debug==True:
      print("blob_ls: container: "+container)
      
    prefix_as_blob_name = prefix[0:-1]
    
    if debug==True:
      print("blob_ls: prefix_as_blob_name: "+prefix_as_blob_name)
      
    #Prefix is blob if exists and size is not 0
    try:
      prefix_as_blob_name_blob_size = block_blob_service.get_blob_properties(container, prefix_as_blob_name).properties.content_length    
      prefix_is_blob_name = block_blob_service.exists(container,prefix_as_blob_name) and (prefix_as_blob_name_blob_size!=0)
      if debug==True:
        print("blob_ls: prefix_is_blob_name: "+str(prefix_is_blob_name))
        print("blob_ls: prefix_is_blob_size: "+str(prefix_as_blob_name_blob_size))
    except Exception as e:
      prefix_is_blob_name = False
      if debug==True:
        print("blob_ls: prefix_is_blob_name: "+str(prefix_is_blob_name))
    
    prefix_is_pattern = (('*' in prefix_as_blob_name)==True) or (('?' in prefix_as_blob_name)==True)
    
    if debug==True:
      print("blob_ls: prefix_is_pattern:" + str(prefix_is_pattern))
      
    list_generator_prefix = []
    #If no container is defined prefix will be '/' or ''
    if(prefix!='' and prefix!='/'):
      if debug==True:
        print("blob_ls: prefix: "+prefix)
        
      #If we are listing 
      if prefix_is_blob_name==True or prefix_is_pattern==True:
        if debug==True:
          print("blob_ls: prefix_is_blob_name: "+str(prefix_is_blob_name))
        prefix = prefix_as_blob_name.replace((prefix_as_blob_name.split('/'))[-1],'')
          
        if debug==True:
          print("blob_ls: Root Prefix: "+prefix)
          
        #List blobs from container with defined folder==path 
        if(len(prefix)!=0):
          if debug==True:
            print("blob_ls: Prefix not Empty")
          generator = block_blob_service.list_blobs(container,prefix=prefix,delimiter='/')
          list_generator = [blob for blob in generator]
        else:
          if debug==True:
            print("blob_ls: Empty Prefix")
          generator = block_blob_service.list_blobs(container,delimiter='/')
          list_generator = [blob for blob in generator if fnmatch(blob.name,prefix_as_blob_name)==True and type(blob).__name__=='Blob']
          list_generator_prefix = [blob for blob in generator if fnmatch(blob.name,prefix_as_blob_name)==True and type(blob).__name__!='Blob']
      else:
        #List blobs from container with defined folder==path
        if debug==True:
          print("blob_ls: Not prefix_is_blob_name==True or prefix_is_pattern==True")
        generator = block_blob_service.list_blobs(container,prefix=prefix)
        list_generator = [blob for blob in generator]
     
      if prefix_is_pattern==True:
        path = path.replace(path.split('/')[-2]+'/','')

      #Generate Dataframe Row with information of the blobs
      row_result = ([Row(path=path+blob.name.replace(prefix,'')+('/' if blob.properties.content_settings.content_type==None else ''),
                        name=blob.name.replace(prefix,''),
                        size=blob.properties.content_length,
                        last_modified=blob.properties.last_modified,
                        etag=blob.properties.etag
                       ) for blob in list_generator if prefix in blob.name ]
                    +
                    [Row(path=path+blob.name.replace(prefix,'')+'/',
                        name=blob.name.replace(prefix,''),
                        size=0,
                        last_modified=0,
                        etag=''
                       ) for blob in list_generator_prefix if prefix in blob.name ]
                   )   
    else:
      if debug==True:
        print("blob_ls: prefix: "+prefix)
        print("blob_ls: no prefix")
      #List blobs from container
      generator = block_blob_service.list_blobs(container,delimiter='/')
      list_generator = [blob for blob in generator]
      

      #Generate Dataframe Row with information of the blobs
      row_result = ([Row(path=path+blob.name+('/' if blob.properties.content_settings.content_type==None else ''),
                         name=blob.name+('/' if blob.properties.content_settings.content_type==None else ''),
                         size=blob.properties.content_length,
                         last_modified=blob.properties.last_modified,
                         etag=blob.properties.etag
                        ) for blob in list_generator if '/' not in blob.name]
                   )
    return row_result
  except Exception as e:
    raise e
    return []

############################################################################################################################################################
  
def blob_delete_file(path,blob_mount_point_root=__BLOB_MOUNT_POINT_ROOT__,blob_account_name=__BLOB_ACCOUNT_NAME__,blob_account_key=__BLOB_ACCOUNT_KEY__,debug=False):
  """Deletes file from azure blob storage referenced by path
      Parameters:
      path : string
        Path of the file to be deleted.
         examples:
           dbfs:/mnt/blob/landing-prod/pharmatic_testdelete/100002930120170126GSL.TXT'
           /mnt/blob/landing-prod/pharmatic_testdelete/100002930120170126GSL.TXT'
      blob_mount_point_root : string(s)
        Root of the mount points of blobs
          example:
            '/mnt/blob'
      blob_account_name : string
        Account Name to connect to blob storage
      blob_account_key
        Account Key to connect to blob storage
      debug: boolean
        Flag to include debug information or Not

      Return: boolean
        Whether the file is deleted or not

  """
#Who                 When           What
 #Victor Salesa Sanz  06/02/2019    Initial version
################################################################################
  try:
    #Connects to blob service
    block_blob_service = BlockBlobService(account_name=blob_account_name, account_key=blob_account_key)
  
    raw_path = path.replace('dbfs:/','/')
      
    # Normalize with dbfs (All the mounts are there)
    path = 'dbfs:' + raw_path

    # Check if folder root is in BLOB mount points list
    is_in_mount_point = __builtin__.max([(1 if raw_path.startswith(mount_point) else 0) for mount_point in __BLOB_MOUNT_POINTS__ ])

    # Check if path contains blob mount point and creates a variable to retrieve container and prefix
    if is_in_mount_point==1:
      raw_path_container_blobs = raw_path.replace(blob_mount_point_root,'')
    else:
      raise Exception("blob_delete_file: Path does not contain a blob connection: "+path+",should be one of: ["+(",".join(__BLOB_MOUNT_POINTS__)) +"]" )

    # In previous string with container and prefix splits by '/' symbol to detect if a container name or prefix is present. 
    split_container_prefix  = raw_path_container_blobs.split('/')

    # Raises an exception if no container defined and extracts it if present
    if len(split_container_prefix) >= 1 and len(split_container_prefix[0])>0:
      # First value splitted by '/' should be container
      container = split_container_prefix[0]
      if debug==True:
        print("blob_delete_file: container: "+container)
    else:
      raise Exception("blob_delete_file: No container defined in the path:"+path)

    #Gets the prefix by removing from raw path the base_blob_path and container
    blob_name = raw_path.replace(blob_mount_point_root+container,'')

    #Removes the leading '/' symbol to match blob service standard
    if blob_name[0]=='/':
      blob_name = blob_name[1:]
      
    #Check if the blob_name exists 
    if(block_blob_service.exists(container,blob_name)==True):
      if debug==True:
        print("blob_delete_file: deleting file: "+(blob_name.split('/'))[-1] )
        print("blob_delete_file: blob_name: "+blob_name)
      block_blob_service.delete_blob(container,blob_name)
    else:
      raise Exception("blob_delete_file: File path doesn't exist: "+path)
    return True
  except Exception as e:
    raise e
    return False

####Changes 21/02/2019########################################################################################################################################################
  
def blob_delete_file_sql_lambda(path):
  """Encapsulates delete file call to avoid exception
    Parameters:
    path : string
      Path of the file to be deleted.
       examples:
         dbfs:/mnt/blob/landing-prod/pharmatic_testdelete/100002930120170126GSL.TXT'
         /mnt/blob/landing-prod/pharmatic_testdelete/100002930120170126GSL.TXT'
    Return: Integer
      Whether the file is deleted or not
  """
#Who                 When           What
#Victor Salesa Sanz  21/02/2019    Initial version
################################################################################
  try:
    ret_value = blob_delete_file(path)
    if ret_value != None:
      return 1
    else:
      return 0
  except Exception as e:
    return 0
  
#register function for %phyton %scala  
blob_delete_file_sql = udf(lambda path: blob_delete_file_sql_lambda(path),IntegerType())

#register function for %sql
spark.udf.register("blob_delete_file_sql",lambda path: blob_delete_file_sql_lambda(path),IntegerType())

############################################################################################################################################################
  
################################################################################################################################################################################################################################################

def GenerateBlobCopyProperties(source_file,dest_file,blob_mount_point_root=__BLOB_MOUNT_POINT_ROOT__, blob_account_name=__BLOB_ACCOUNT_NAME__, blob_account_key=__BLOB_ACCOUNT_KEY__,method_name='',debug=False):
  """Generates a BlobCopyProperties object with container_source/source_blob_name pair and destination_blob/container_dest pair. As blob doesn't permit move directly but copy and delete. We will use this in both functions.
  
      Parameters:
      source_file : string
        Source file path.
         examples:
           'dbfs:/mnt/blob/landing/pharmatic/100002930120170126GPR.TXT'
           '/mnt/blob/landing/pharmatic/100002930120170126GPR.TXT'
      destination_file: string
        Destination file path. Could be a single folder or a file
         examples:
           'dbfs:/mnt/blob/landing/pharmatic/100002930120170126GPR_RENAMED.TXT'
           '/mnt/blob/landing/pharmatic/100002930120170126GPR_RENAMED.TXT'
      
      blob_mount_point_root : string(s)
        Root of the mount points of blobs
          example:
            '/mnt/blob'
      blob_account_name : string
        Account Name to connect to blob storage
      blob_account_key
        Account Key to connect to blob storage
      method_name:
        As this is a util function we need to know the caller function name for debugging and error control
      debug: boolean
        Flag to include debug information or Not
      
      Return:  BlobCopyProperties object with values to copy between blobs
        
  """
  #Who                 When           What
 #Victor Salesa Sanz  08/02/2019    Initial version
 #Victor Salesa Sanz  21/03/2019    Added prefix folder checking in copy properties process
 #Victor Salesa Sanz  06/05/2019    Removed exception when file is found in dest with  bytes <>0 
################################################################################
  try:

    #Connects to blob service
    block_blob_service = BlockBlobService(account_name=blob_account_name, account_key=blob_account_key)

    raw_path_source =  source_file.replace('dbfs:/','/')
    raw_path_dest   =  dest_file.replace('dbfs:/','/')

    # Normalize with dbfs (All the mounts are there)
    source_file   = 'dbfs:' + raw_path_source
    dest_file     = 'dbfs:' + raw_path_dest

    # Check if folder root is in BLOB mount points list
    is_in_mount_point_source = __builtin__.max([(1 if raw_path_source.startswith(mount_point) else 0) for mount_point in __BLOB_MOUNT_POINTS__ ])
    is_in_mount_point_dest   = __builtin__.max([(1 if raw_path_dest.startswith(mount_point) else 0) for mount_point in __BLOB_MOUNT_POINTS__ ])

    # Check if source_file path contains blob mount point and creates a variable to retrieve container and prefix
    if is_in_mount_point_source==True :
      raw_path_container_blobs_source = raw_path_source.replace(blob_mount_point_root,'')
    else:
      raise Exception(method_name + " Path does not contain a blob connection: "+source_file+",should be one of: ["+(",".join(__BLOB_MOUNT_POINTS__)) +"]" )

    # Check if dest_file path contains blob mount point and creates a variable to retrieve container and prefix
    if is_in_mount_point_dest==True :
      raw_path_container_blobs_dest = raw_path_dest.replace(blob_mount_point_root,'')
    else:
      raise Exception(method_name + " Path does not contain a blob connection: "+source_file+",should be one of: ["+(",".join(__BLOB_MOUNT_POINTS__)) +"]" )

    # In previous string with container and prefix splits by '/' symbol to detect if a container name or prefix is present. 
    split_container_prefix_source  = raw_path_container_blobs_source.split('/')
    split_container_prefix_dest  = raw_path_container_blobs_dest.split('/')

    # Raises an exception if no container defined for source file and extracts it if present
    if len(split_container_prefix_source) >= 1 and len(split_container_prefix_source[0])>0:
      # First value splitted by '/' should be container
      source_container = split_container_prefix_source[0]
      if debug==True:
        print(method_name + " source_container: "+source_container)
    else:
      raise Exception(method_name + " No container defined in the path:"+source_file)

    # Raises an exception if no container defined for source file and extracts it if present
    if len(split_container_prefix_dest) >= 1 and len(split_container_prefix_dest[0])>0:
      # First value splitted by '/' should be container
      dest_container = split_container_prefix_dest[0]
      if debug==True:
        print(method_name + " dest_container: "+dest_container)
    else:
      raise Exception(method_name + " No container defined in the path:"+dest_file)
    if debug==True:
      print(method_name + " blob_mount_point_root+source_container: ",blob_mount_point_root+source_container)
      print(method_name + " raw_path_source: "+raw_path_source)  
    #Gets the prefix by removing from raw path the base_blob_path and container
    source_blob_name = raw_path_source.replace(blob_mount_point_root+source_container,'')
    if debug==True:
      print(method_name + " blob_mount_point_root+dest_container: ",blob_mount_point_root+dest_container)
      print(method_name + " raw_path_dest: "+raw_path_dest)
    #Gets the prefix by removing from raw path the base_blob_path and container
    dest_blob_name = raw_path_dest.replace(blob_mount_point_root+dest_container,'')

    #Removes the leading '/' symbol to match blob service standard
    if source_blob_name[0]=='/':
      source_blob_name = source_blob_name[1:]

    #Removes the leading '/' symbol to match blob service standard
    if dest_blob_name[0]=='/':
      dest_blob_name = dest_blob_name[1:]
    if debug==True:
      print(method_name + " source_blob_name: "+source_blob_name)
      print(method_name + " dest_blob_name: "+dest_blob_name)
  
    #Check if source file exists
    try:
      source_blob_name_blob_size = block_blob_service.get_blob_properties(source_container, source_blob_name).properties.content_length 
    except Exception as e:
      source_blob_name_blob_size = 0
      
    ####Source file Checkings

    #Check if source file exists
    try:
      source_blob_name_blob_size = block_blob_service.get_blob_properties(source_container, source_blob_name).properties.content_length 
    except Exception as e:
      source_blob_name_blob_size = 0

    #If source file exists and destination is not a "FileName" but a "path" create a "prefix-filename" with destination prefix
    source_blob_exists = block_blob_service.exists(source_container,source_blob_name)
    if(source_blob_exists and source_blob_name_blob_size!=0):
      file_name = (source_blob_name.split('/'))[-1]
    else:
      if source_blob_exists==False:
        raise Exception(method_name+ ": Source File Is Empty or Not Exists: container:"+source_container+",source_blob_name:"+source_blob_name)
      else:
        file_name = (source_blob_name.split('/'))[-1]

    # ####Destination Folder Checkings  

    #Check if folder name ends with '/' to remove it and check if it's a folder
    if(dest_blob_name[-1:]=='/'):
        dest_blob_name_subfolders_count = len(dest_blob_name.split('/'))
        dest_blob_to_check = ('/').join((dest_blob_name.split('/'))[:-1])
        separator = ''
    else:
        dest_blob_to_check = dest_blob_name
        separator = '/'
    if debug==True:
      print(method_name+": DestBlobtoCheck"+dest_blob_to_check)

    #If destination is a "folder" then create a folder/source_file_name structure blob name
    if block_blob_service.exists(dest_container,dest_blob_to_check)==True:
      dest_blob_to_check_object = block_blob_service.get_blob_properties(dest_container,dest_blob_to_check)
      if  type(dest_blob_to_check_object).__name__ == "Blob":
        if dest_blob_to_check_object.properties.content_length == 0:
          if debug==True:
            print(method_name+": Is Folder empty")
          dest_blob_name = dest_blob_name+separator+file_name
      else: 
        if type(dest_blob_to_check_object).__name__ == "BlobPrefix":
          if debug==True:
            print(method_name+": Is Folder prefix")
          dest_blob_name = dest_blob_name+separator+file_name
        else:
          if debug==True:
            print(method_name+": Is Not a Folder")
    else:
      #----- If destination is a Folder Prefix then create a destination folder/source_file_name structure blob name
      prefixes = [blob.name.replace('/','') for blob in list(block_blob_service.list_blobs(dest_container,delimiter='/')) if type(blob).__name__=='BlobPrefix'] 
      dest_blob_to_check_is_prefix = dest_blob_to_check in prefixes  
      if dest_blob_to_check_is_prefix == True:
        if debug==True:
          print(method_name+": Is Folder Prefix " + dest_blob_to_check)
        dest_blob_name = dest_blob_name+separator+file_name
      else:
        if debug==True:
          print(method_name+": Is Not a Folder")
      #-----

    copy_properties = BlobCopyProperties(source_container=source_container,dest_container=dest_container,source_blob_name=source_blob_name,dest_blob_name=dest_blob_name)
    return copy_properties
  
  except Exception as e:
    raise e
    
################################################################################################################################################################################################################################################

def blob_copy_file(source_file,dest_file,blob_mount_point_root=__BLOB_MOUNT_POINT_ROOT__, blob_account_name=__BLOB_ACCOUNT_NAME__, blob_account_key=__BLOB_ACCOUNT_KEY__,debug=False,timeout=1):
  """Copy File from source file to destination_file
      Parameters:
      source_file : string
        Source file path.
         examples:
           'dbfs:/mnt/blob/landing/pharmatic/100002930120170126GPR.TXT'
           '/mnt/blob/landing/pharmatic/100002930120170126GPR.TXT'
      destination_file: string
        Destination file path. Could be a single folder or a file
         examples:
           'dbfs:/mnt/blob/landing/pharmatic/100002930120170126GPR_RENAMED.TXT'
           '/mnt/blob/landing/pharmatic/100002930120170126GPR_RENAMED.TXT'
      
      blob_mount_point_root : string(s)
        Root of the mount points of blobs
          example:
            '/mnt/blob'
      blob_account_name : string
        Account Name to connect to blob storage
      blob_account_key
        Account Key to connect to blob storage
      method_name:
        As this is a util function we need to know the caller function name for debugging and error control
      timeout: int
        Timeout expressed in seconds
      debug: boolean
        Flag to include debug information or Not
      
      Return:  BlobCopyProperties object with values to copy between blobs
        
  """
  #Who                 When           What
 #Victor Salesa Sanz  08/02/2019    Initial version
 #Ana Perez           01/03/2019    Fix issue in blob_copy_file_sql register
 #Victor Salesa Sanz  21/03/2019    Assure that blob is copied returning
 ################################################################################
  try: 
    blob_copy_properties = GenerateBlobCopyProperties(source_file=source_file,dest_file=dest_file,method_name='blob_copy_file')
    block_blob_service = BlockBlobService(account_name=blob_account_name, account_key=blob_account_key)
    source_blob_url = block_blob_service.make_blob_url(blob_copy_properties.source_container, blob_copy_properties.source_blob_name)
    #---------------
    if debug==True:
      print("["+datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')+"]"+" Start Copying Blob")
    copy_result = block_blob_service.copy_blob(blob_copy_properties.dest_container,blob_copy_properties.dest_blob_name, source_blob_url,timeout=timeout)
    copy_status = copy_result.status
    if debug==True:
      print("["+datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')+"]"+" Copy Status: "+copy_status+" for Blob "+blob_copy_properties.dest_blob_name)
    while copy_status!='success' and copy_status!='aborted' and copy_status!='fail':
      copy_status = block_blob_service.get_blob_properties(blob_copy_properties.dest_container,blob_copy_properties.dest_blob_name).properties.copy.status
      if debug==True:
        print("["+datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')+"]"+" Copy Status: "+copy_status+" for Blob "+blob_copy_properties.dest_blob_name)
    if copy_status=='success' and block_blob_service.exists(blob_copy_properties.dest_container,blob_copy_properties.dest_blob_name)==True:
      return True
    else:
      return False
    #---------------
  except Exception as e:
    raise e
    
#register function for %sql
spark.udf.register("blob_copy_file_sql",lambda src_file,dest_file: blob_copy_file (src_file,dest_file),BooleanType())
#register function for %phyton %scala
blob_copy_file_sql = udf(lambda src_file,dest_file: blob_copy_file (src_file,dest_file),BooleanType())
    
################################################################################################################################################################################################

def blob_move_file(source_file,dest_file,blob_mount_point_root=__BLOB_MOUNT_POINT_ROOT__, blob_account_name=__BLOB_ACCOUNT_NAME__, blob_account_key=__BLOB_ACCOUNT_KEY__,debug=False,timeout=1):
  """Move File from source file to destination_file
      Parameters:
      source_file : string
        Source file path.
         examples:
           'dbfs:/mnt/blob/landing/pharmatic/100002930120170126GPR.TXT'
           '/mnt/blob/landing/pharmatic/100002930120170126GPR.TXT'
      destination_file: string
        Destination file path. Could be a single folder or a file
         examples:
           'dbfs:/mnt/blob/landing/pharmatic/100002930120170126GPR_RENAMED.TXT'
           '/mnt/blob/landing/pharmatic/100002930120170126GPR_RENAMED.TXT'
      
      blob_mount_point_root : string(s)
        Root of the mount points of blobs
          example:
            '/mnt/blob'
      blob_account_name : string
        Account Name to connect to blob storage
      blob_account_key
        Account Key to connect to blob storage
      method_name:
        As this is a util function we need to know the caller function name for debugging and error control
      timeout: int
        Timeout expressed in seconds
      debug: boolean
        Flag to include debug information or Not
      
      Return:  BlobCopyProperties object with values to copy between blobs
        
  """
  #Who                 When           What
 #Victor Salesa Sanz  08/02/2019    Initial version
 #Ana Perez           01/03/2019    Fix issue in blob_move_file_sql register
 #Victor Salesa Sanz  21/03/2019    Assure that blob is copied before deleting 
################################################################################
  try:
    blob_copy_properties = GenerateBlobCopyProperties(source_file=source_file,dest_file=dest_file,method_name='blob_move_file')
    block_blob_service = BlockBlobService(account_name=blob_account_name, account_key=blob_account_key)
    source_blob_url = block_blob_service.make_blob_url(blob_copy_properties.source_container, blob_copy_properties.source_blob_name)
    #---------------
    if debug==True:
      print("["+datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')+"]"+" Start Copying Blob")
    copy_result = block_blob_service.copy_blob(blob_copy_properties.dest_container,blob_copy_properties.dest_blob_name, source_blob_url,timeout=timeout)
    copy_status = copy_result.status
    if debug==True:
      print("["+datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')+"]"+" Copy Status: "+copy_status+" for Blob "+blob_copy_properties.dest_blob_name)
    while copy_status!='success' and copy_status!='aborted' and copy_status!='fail':
      copy_status = block_blob_service.get_blob_properties(blob_copy_properties.dest_container,blob_copy_properties.dest_blob_name).properties.copy.status
      if debug==True:
        print("["+datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')+"]"+" Copy Status: "+copy_status+" for Blob "+blob_copy_properties.dest_blob_name)
    if copy_status=='success' and block_blob_service.exists(blob_copy_properties.dest_container,blob_copy_properties.dest_blob_name)==True:
      #If the file was moved successfully delete it from source
      print("["+datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')+"]"+"Deleting Blob "+blob_copy_properties.source_blob_name)
      block_blob_service.delete_blob(blob_copy_properties.source_container,blob_copy_properties.source_blob_name)
      return True
    else:
      return False
    #---------------
  except Exception as e:
    raise e

#register function for %sql
spark.udf.register("blob_move_file_sql",lambda src_file,dest_file: blob_move_file(src_file,dest_file),BooleanType())
#register function for %phyton %scala
blob_move_file_sql = udf(lambda src_file,dest_file: blob_move_file(src_file,dest_file),BooleanType())